# Example 3

- The k-out-of-n system
    - The system consists of $n$ **non-identical** compoments.
    - The system fails when $k$ compoments are down.
- Repairable system
    - A failed compoment may be repaired.
    - The repaired component is as good as new.
- Repairperson
    - Failed components are repaired by a single repairperson.
    - A failed component makes a queue with **a random** discipline
- The failure time of **i-th component** follows an exponential distribution with mean $1/\lambda_i$.
- The repair time for a failed component follows an exponential distribution with mean $1/\mu$.

## Initialize

Load packages

In [ ]:
using Origin
using SparseMatrix
using NMarkov
using JuliaDot
using SparseArrays
using Plots
using MAT
using JSON
using Distributions
using Random

In [ ]:
# utility functions

function drawfile(x)
    data = open(x) do f
        read(f, String)
    end
    draw(data) # JuliaDot.draw
end

import NMarkov.eye
function eye(M::AbstractMatrix)
    eye(size(M)[1])
end

function speye(M::AbstractMatrix)
    n = size(M)[1]
    x = [i for i = 1:n]
    v = [1.0 for i = 1:n]
    sparse(x, x, v)
end

## Build the model with SPN

1. Make the SPN definition file (reliab2.spn)
1. Draw a diagram of SPN
1. Check the model behavior with one simulation

### Draw a petrinet diagram

In [ ]:
# generate a dot file to draw PN with gospn
run(`./gospn view -i reliab2.spn -o tmp.dot`)
# draw a picture with the dot file
drawfile("tmp.dot")

### Check the model behavior

In [ ]:
# set model params
rng = MersenneTwister(1234)
lambda = rand(rng, Uniform(0.01, 0.02), 10) # different failure rates
mu = 1.0
paramstring = join(["lambda$i = $(lambda[i]); " for i = 1:10]) * "mu = $mu;"

In [ ]:
# test the SPN model to check 30 transitions from the initial marking
run(`./gospn test -i reliab2.spn -n 30 -p "$paramstring"`)

## Construct CTMC matrices

1. Analyze SPN and generate the following files
    - A Matlab matrix file to store matrices (option: -o)
    - A dot file to draw the marking graph (reachability graph). In the case where SPN generates a large number of states, it is better not to generate this file (option: -m)
    - A dot file to draw the transitions between marking groups (option: -g)
2. Construct CTMC matrices from the matrix file and a diagrm of marking group

### Generate the marking graph

In [ ]:
# generate CTMC matricies, dot files for marking and group
run(`./gospn mark -i reliab2.spn -o result.mat -g gmark.dot -p "$paramstring"`)
# read CTMC matrices
matfile = matopen("result.mat") 
# draw a picture of groups of markings
drawfile("gmark.dot")

In [ ]:
# Read matrices
G0G0E = read(matfile, "G0G0E")
G0I0E = read(matfile, "G0I0E")
I0G0I = read(matfile, "I0G0I");

In [ ]:
Q = G0G0E + G0I0E * I0G0I;

## Compute reliability measures
- Steady-state analysis
    1. Compute the steady-state probability vector
    2. Compute the steady-state reward by multiplying the steady-state probability vector with a reward vector
- Transient analysis
    1. Compute the reward for each time point

In [ ]:
# reward vector
r = read(matfile, "availG0");

In [ ]:
# steady-state vector
pis, conv = stgs(Q) # GS algorithm for sparse matrix

In [ ]:
savail = sum(pis .* r) # system availability

In [ ]:
ts = LinRange(0, 10000, 10000) # time points
x0 = read(matfile, "initG0"); # initial probability vector

In [ ]:
# transient analysis
#   irwd: instantaneous reward
#   crwd: cumulative reward
#   xt: the state probability vector at the end of ts
#   cxt: the cumulative state probability vector at the end of ts
irwd, crwd, xt, cxt = tran(Q, x0, r, ts, forward=:T)

In [ ]:
plot(ts, irwd)

In [ ]:
v = findall(x->x==1, r) # indices in which the system is working
T = Q[v,v] # submatrix
inita = x0[v]
rwda = r[v];

In [ ]:
# transient analysis
#   irwd: instantaneous reward
#   crwd: cumulative reward
#   xt: the state probability vector at the end of ts
#   cxt: the cumulative state probability vector at the end of ts
irwd, crwd, xt, cxt = tran(T, inita, rwda, ts, forward=:T)

In [ ]:
plot(ts, irwd)